This version uses Milvus through Docker Compose so you must have Docker installed to run this notebook (Milvus is spun up via `docker compose up -d` as shown in the block below)

In [ ]:
# ! pip install pymilvus milvus langchain sentence-transformers tiktoken octoai-sdk
# docker-compose up -d

In [1]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OCTOAI_TOKEN"] = os.getenv("OCTOAI_API_TOKEN")

In [3]:
llm = OctoAIEndpoint(
        model="mixtral-8x7b-instruct-fp16",
        max_tokens=200,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )

In [4]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.vectorstores import Milvus

In [5]:
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

In [6]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import os

In [7]:
files = os.listdir("./data")

In [8]:
files

['Chicago.txt',
 'Washington, D.C..txt',
 'Cambridge, Massachusetts.txt',
 'Houston.txt',
 'Seattle.txt',
 'Toronto.txt',
 'San Francisco.txt',
 'Boston.txt']

In [9]:
file_texts = []

In [10]:
for file in files:
    with open(f"./data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512, chunk_overlap=64, 
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

Created a chunk of size 1299, which is longer than the specified 512
Created a chunk of size 597, which is longer than the specified 512
Created a chunk of size 524, which is longer than the specified 512
Created a chunk of size 535, which is longer than the specified 512
Created a chunk of size 843, which is longer than the specified 512
Created a chunk of size 746, which is longer than the specified 512
Created a chunk of size 887, which is longer than the specified 512
Created a chunk of size 662, which is longer than the specified 512
Created a chunk of size 515, which is longer than the specified 512
Created a chunk of size 615, which is longer than the specified 512
Created a chunk of size 875, which is longer than the specified 512
Created a chunk of size 807, which is longer than the specified 512
Created a chunk of size 730, which is longer than the specified 512
Created a chunk of size 992, which is longer than the specified 512
Created a chunk of size 793, which is longer th

In [11]:
file_texts[0]

Document(page_content="Chicago (  shih-KAH-goh, locally also  shih-KAW-goh; Miami-Illinois: Shikaakwa; Ojibwe: Zhigaagong) is the most populous city in the U.S. state of Illinois and the third-most populous in the United States after New York City and Los Angeles. With a population of 2,746,388 in the 2020 census, it is also the most populous city in the Midwest. As the seat of Cook County, the second-most populous county in the U.S., Chicago is the center of the Chicago metropolitan area.\nLocated on the shore of Lake Michigan, Chicago was incorporated as a city in 1837 near a portage between the Great Lakes and the Mississippi River watershed. It grew rapidly in the mid-19th century. In 1871, the Great Chicago Fire destroyed several square miles and left more than 100,000 homeless, but Chicago's population continued to grow. Chicago made noted contributions to urban planning and architecture, such as the Chicago School, the development of the City Beautiful Movement, and the steel-fr

In [12]:
vector_store = Milvus.from_documents(
    file_texts,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": 19530},
    collection_name="cities"
)

In [13]:
retriever = vector_store.as_retriever()

In [14]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [15]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
chain.invoke("How big is Seattle?")

" Seattle is a significant city with a 2022 population of 749,256 people, making it the most populous city in both the state of Washington and the Pacific Northwest region of North America. The Seattle metropolitan area's population is 4.02 million, making it the 15th-largest in the United States."

In [ ]:
# Let's make this a bit more fun and showcase the multilingual capabilities of Mixtal which really outshine other open source models

# Our Vector DB is populated with entries from english text - even the embedding model we're using here, GTE-Large
# works best on english text. However Mixtral has good mutlilingual capabilities in French, German, Spanish and Italian.
# So what we'll do is ask the assistant to only answer in french in the system and user prompt. RAG here is performed based on 
# english text, but upon producing the user response, the Mixtral LLM will generate tokens in a different language here (french)
french_llm = OctoAIEndpoint(
        model="mixtral-8x7b-instruct-fp16",
        max_tokens=200,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )

french_template = """Answer the question in French based only on the following context:
{context}

Question: {question}
"""
french_prompt = PromptTemplate.from_template(french_template)

In [ ]:
french_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | french_prompt
    | french_llm
    | StrOutputParser()
)

In [ ]:
fr_1 = french_chain.invoke("How big is the city of Seattle?")

In [ ]:
from pprint import pprint
pprint(fr_1)

In [ ]:
fr_2 = french_chain.invoke("When should I visit Seattle?")
pprint(fr_2)

In [ ]:
fr_3 = french_chain.invoke("What are some things to do Seattle?")
pprint(fr_3)

In [ ]:
english_chain = LLMChain(prompt=prompt, llm=llm)

template = """Translate the entry into english {entry} """
prompt = PromptTemplate.from_template(template)

print(llm_chain.invoke(fr_1)["text"])